#  Web scraping and Data cleaning on an E-Commerce Website

With the evolution of the internet, e-commerce has become so popular that now virtually everyone purchases one or the other product online. With just a few clicks one can have their wish product delivered at their doorstep. In spite of being so convenient, it can be exasperating at times, mainly attributing to the numerous alternatives available on various e-commerce platforms.

Through this project, we are trying to assist one of our customer who wants to purchase a laptop. His hectic work and personal life do not allow him to spend enough time to go through these websites and finalize his purchase. He was in search of support who can help him to finalize on his wish laptop. This brought him to us who gladly accepted this offer.

![e-commerce](e_commerce.png)

[Image source ](https://www.freepik.com/search?dates=any&format=search&page=1&query=online+shopping&sort=popular)

To assist him, we agreed to scrap the data from amazon.com. We need to do some data cleaning before we can present him with our final list based on his requirements. He can then use this to make his final call. 
In this project, we will be working on web scraping and data cleaning.

The Customer Requirements:
 - Processor : Intel i5 or i7
 - RAM : minimum 8GB 
 - Screen size : between 14 and 16
 - Price limit : upto $1500
 - Good user ratings : 4 and above on 5 star rating system

Now we have all that is needed to start our business, let's begin.

# The Approach

- scraping the required data from an e-commerce website, for this project, it is from amazon.com
- cleaning the data to enable easier analysis
- analyzing the data based on the customer requirements
 
# Initial Setup

We will start with our initial set up where we will import the libraries required for this project.

In [907]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Scraping the Data

Here we will scrap relevant information from first the five pages of our search result, which is for laptops from amazon.com as that would give us enough data for our analysis. 
In this step, we are creating a data frame from the scrapped data. We can also create an excel file.

In [896]:
all = []
for pageNo in range(1,6):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
               "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
               "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    req = requests.get('https://www.amazon.com/s?k=laptops&page='+str(pageNo), headers=headers)
    content = req.content
    soup = BeautifulSoup(content, 'lxml') 

    for d in soup.findAll('div', attrs={'class':'sg-col-4-of-12 sg-col-8-of-16 sg-col-16-of-24 sg-col-12-of-20 sg-col-24-of-32 sg-col sg-col-28-of-36 sg-col-20-of-28'}):
        name_span = d.find('span', attrs={'class':'a-size-medium a-color-base a-text-normal'})
        price_span = d.find('span', attrs={'class':'a-offscreen'})
        rating_span = d.find('span', attrs={'class':'a-icon-alt'})

        if name_span is not None:
            name = name_span.text
        else:
            name = 'unknown product'  
            
        if price_span is not None:
            price = price_span.text
        else:
            price = 'unknown price'
            
        if rating_span is not None:
            rating = rating_span.text
        else:
            rating = 'unknown rating'
            
        all.append([name, price, rating])
        
laptops_info = pd.DataFrame(all,columns=['name', 'price_$', 'ratings'])    
laptops_info

,name,price_$,ratings
0,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x...",$449.99,3.7 out of 5 stars
1,"LG Gram 2-in-1 Convertible Laptop: 14"" Full HD...","$1,596.99",4.4 out of 5 stars
2,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",$349.99,4.4 out of 5 stars
3,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",$579.99,4.4 out of 5 stars
4,"Acer Spin 3 Convertible Laptop, 14 inches Full...",$799.99,4.3 out of 5 stars
...,...,...,...
105,2020 Dell Inspiron 3000 15.6-inch HD Touchscre...,$477.90,unknown rating
106,"Newest HP 14 Laptop, 14"" Widescreen LED, AMD A...",$379.99,4.0 out of 5 stars
107,Lenovo 100E Chromebook 2ND Gen 81QB000AUS Lapt...,$230.00,4.1 out of 5 stars
108,"ASUS VivoBook L203MA Ultra-Thin Laptop, 11.6” ...",$269.99,4.1 out of 5 stars


# Cleaning the dataset

Before we start analyzing, the dataset needs to be cleaned. Clean data will help us to analyze faster.

In [897]:
# Understading the data types of the dataset
laptops_info.dtypes

name       object
price_$    object
ratings    object
dtype: object

# Cleaning the price column

One of the requirements provided by the customer is the price limit. To make this analysis we need to clean the price column. We will perform the below cleaning operations:
- remove the rows which don't provide price information as the customer is not interested in such options
- remove the '\\$' sign as this information is clear from the column name itself
- convert the column into a numeric type and then round the values which makes it easier for analysis

In [898]:
# Identifying different values in price column
laptops_info['price_$'].value_counts(dropna=False)

unknown price    13
$269.99           9
$999.99           7
$449.99           6
$579.99           6
                 ..
$249.99           1
$287.00           1
$399.00           1
$11.37            1
$477.90           1
Name: price_$, Length: 61, dtype: int64

In [899]:
# Removing the rows with unknown price
laptops_info = laptops_info[laptops_info['price_$']!='unknown price']

# Function to clean the price column
def clean_price(price):
    price = price.replace('$', '')  # Replace the '$' with blank eg: '$1,249.99' to '1,249.99'
    price = price.replace(',', '')  # Replace the ',' with blank eg: '1,249.99' to '1249.99'
    price = pd.to_numeric(price)    # Convert the data type to numeric
    price = price.round()           # Round the values
    return price

# Create a new column with the cleaned price
pd.set_option('mode.chained_assignment',None) # to suppress 'SettingWithCopyWarning' warnings
laptops_info['final_price_$'] = laptops_info['price_$'].apply(clean_price)

# Remove entries where price is less than 100 as these could be accessories
laptops_info = laptops_info[laptops_info['final_price_$']>=100]
laptops_info.head()

,name,price_$,ratings,final_price_$
0,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x...",$449.99,3.7 out of 5 stars,450.0
1,"LG Gram 2-in-1 Convertible Laptop: 14"" Full HD...","$1,596.99",4.4 out of 5 stars,1597.0
2,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",$349.99,4.4 out of 5 stars,350.0
3,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",$579.99,4.4 out of 5 stars,580.0
4,"Acer Spin 3 Convertible Laptop, 14 inches Full...",$799.99,4.3 out of 5 stars,800.0


# Cleaning the rating column

The customer wishes to invest in a laptop which is quite popular in the market. This can be analyzed using user ratings. The rating column is having values such as 4.5 out of 5 stars. What is important here for our analysis is the value of 4.5. We will perform the below cleaning operations on this column.
- extract only the rating
- convert the column into a numeric type which makes it easier to analyze
- replace unknown rating with 0, since removing this row completely may not be a good idea as the customer in spite having rating being 0 still interested to have a look at it

In [900]:
# Identifying different values in ratings column
laptops_info['ratings'].value_counts(dropna=False)

4.0 out of 5 stars    20
4.4 out of 5 stars    15
4.1 out of 5 stars    12
4.2 out of 5 stars     9
3.7 out of 5 stars     7
4.3 out of 5 stars     6
3.9 out of 5 stars     6
unknown rating         5
4.5 out of 5 stars     4
4.6 out of 5 stars     4
3.8 out of 5 stars     3
3.6 out of 5 stars     2
3.5 out of 5 stars     2
3.0 out of 5 stars     1
Name: ratings, dtype: int64

In [901]:
# Function to clean ratings column
def ratings_clean(rating):
    rating = rating.split()
    rating = rating[0]
    if rating == 'unknown':
        rating = 0
    rating = pd.to_numeric(rating) # Convert the data type to numeric
    return rating

# Create a new column with the cleaned ratings
laptops_info['ratings_clean'] = laptops_info['ratings'].apply(ratings_clean)
laptops_info.head()

,name,price_$,ratings,final_price_$,ratings_clean
0,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x...",$449.99,3.7 out of 5 stars,450.0,3.7
1,"LG Gram 2-in-1 Convertible Laptop: 14"" Full HD...","$1,596.99",4.4 out of 5 stars,1597.0,4.4
2,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",$349.99,4.4 out of 5 stars,350.0,4.4
3,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",$579.99,4.4 out of 5 stars,580.0,4.4
4,"Acer Spin 3 Convertible Laptop, 14 inches Full...",$799.99,4.3 out of 5 stars,800.0,4.3


# Extracting more information from the name column

If you remember, our customer's first three requirements were related to technical details. The name column consists of a lot of information about the product along with technical details such as the brand name, screen size, memory, and so on. The first three technical requirements provided by our customer is also hidden in this column. So to make an effective analysis we need to extract these pieces of information. Our next step consists of this process. This is the most complicated part of this project. For extraction, we need to make use of regular expression commonly known as regex.

First, we will extract the processor details and create a new column with this information.

In [902]:
# Extract processor details
reg_processor = r'\b([iI][\d])\b'
laptops_info['processor'] = laptops_info['name'].str.extract(reg_processor)
laptops_info.head()

,name,price_$,ratings,final_price_$,ratings_clean,processor
0,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x...",$449.99,3.7 out of 5 stars,450.0,3.7,NaN
1,"LG Gram 2-in-1 Convertible Laptop: 14"" Full HD...","$1,596.99",4.4 out of 5 stars,1597.0,4.4,i7
2,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",$349.99,4.4 out of 5 stars,350.0,4.4,NaN
3,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",$579.99,4.4 out of 5 stars,580.0,4.4,i5
4,"Acer Spin 3 Convertible Laptop, 14 inches Full...",$799.99,4.3 out of 5 stars,800.0,4.3,i7


Next we will extract the RAM details and create a new column with this information.

In [903]:
# Extract RAM details    
reg_RAM = r'\b([\d]+)[GB]+[ ][\+ LlPpDdRrAaMmEeOoYy\d]+\b' # Caputure group is only digits
laptops_info['ram_gb'] = laptops_info['name'].str.extract(reg_RAM)

# Function to clean ram column
def ram_clean(ram):
    ram = pd.to_numeric(ram)
    return ram

laptops_info['ram_gb'] = laptops_info['ram_gb'].apply(ram_clean)
laptops_info.head()

,name,price_$,ratings,final_price_$,ratings_clean,processor,ram_gb
0,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x...",$449.99,3.7 out of 5 stars,450.0,3.7,NaN,8.0
1,"LG Gram 2-in-1 Convertible Laptop: 14"" Full HD...","$1,596.99",4.4 out of 5 stars,1597.0,4.4,i7,16.0
2,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",$349.99,4.4 out of 5 stars,350.0,4.4,NaN,4.0
3,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",$579.99,4.4 out of 5 stars,580.0,4.4,i5,8.0
4,"Acer Spin 3 Convertible Laptop, 14 inches Full...",$799.99,4.3 out of 5 stars,800.0,4.3,i7,16.0


Finally we will extract the screen size details and create a new column with this information.

In [904]:
# Extract screen size details
reg_screen = r'\b([1][\d]+[\.IiNnCcHh\d]*)[ \d]*\b' # Caputure group is only digits
laptops_info['screen_size_inches'] = laptops_info['name'].str.extract(reg_screen)

# Function to clean screen size column
def screen_size_clean(size):
    size = size.replace('inch', '')
    size = size.replace('in', '')
    size = pd.to_numeric(size)
    return size

laptops_info['screen_size_inches'] = laptops_info['screen_size_inches'].apply(screen_size_clean)

# Final Analysis

We are now done with our crucial part of this project which consists of collecting data and cleaning them. We are ready to go ahead with our analysis. Before we begin with our final analysis we will have a look at the customer requirements.

The Customer Requirements:
 - Processor : Intel i5 or i7
 - RAM : minimum 8GB 
 - Screen size : between 14 and 16
 - Price limit : upto $1500
 - Good user ratings : 4 and above on 5 star rating system
 
Let's now create the required filtering to create the final dataset.

In [905]:
# columns for the final dataset
columns_final = ['name', 'final_price_$', 'ratings_clean', 'processor', 'ram_gb', 'screen_size_inches']

# Conditions to filter the processor
laptops_final = laptops_info[columns_final][(laptops_info['processor']=='i7') | (laptops_info['processor']=='I7') | 
                             (laptops_info['processor']=='i5') | (laptops_info['processor']=='I5')].copy()

# Conditions to filter the ram and price
laptops_final = laptops_final[(laptops_final['ram_gb']>=8) & (laptops_final['final_price_$']<=1500)]

# Conditions to filter the screen size
laptops_final = laptops_final[(laptops_final['screen_size_inches']>=14) & (laptops_final['screen_size_inches']<=16)]

# Condition to filter the rating
laptops_final = laptops_final[laptops_final['ratings_clean']>=4]
laptops_final

,name,final_price_$,ratings_clean,processor,ram_gb,screen_size_inches
3,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",580.0,4.4,i5,8.0,15.6
4,"Acer Spin 3 Convertible Laptop, 14 inches Full...",800.0,4.3,i7,16.0,14.0
8,"2020 HP 15 15.6"" HD Touchscreen Premium Laptop...",699.0,4.2,i5,16.0,15.0
10,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",580.0,4.4,i5,8.0,15.6
16,"Lenovo Ideapad L340 Gaming Laptop, 15.6 Inch F...",739.0,4.4,i5,8.0,15.6
24,"2020 HP 15.6"" HD Touchscreen Premium Home & Bu...",669.0,4.5,i5,8.0,15.6
33,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",580.0,4.4,i5,8.0,15.6
62,Dell Inspiron 15.6 Inch HD Touchscreen Flagshi...,612.0,4.2,i5,8.0,15.6
65,"Acer Aspire 5 Slim Laptop, 15.6"" Full HD IPS D...",580.0,4.4,i5,8.0,15.6
75,"New HP Pavilion 2-in-1 15.6"" HD Touchscreen La...",694.0,4.2,i5,8.0,15.6


# Conclusion

After our analysis as per the customer requirements, we created a final dataset that consists of very few selections. Now we can hand over our customer this final dataset that would help him finalize his purchase.

In this project, we applied a generic approach on amazon.com which can also be applied to other similar platforms by analyzing it's page layouts. We can also increase the number of pages we need to scrap which in this case was limited to 5. The platform itself gives more options to filter. But the idea of this project was to understand how to perform web scraping, data cleaning, and analyzing. 

I hope you enjoyed this project.

Thank you!